# Imports

In [1]:
import json,os,shutil,yaml,torch
import torch.nn as nn
from models import create_model
from utils.util import get_optimizer, get_loss, get_scheduler
from utils.data_container import get_data_loader
from utils.preprocess import preprocessing_for_metric
from utils.train import train_model, test_model


In [2]:
def train(conf, data_category):
    print(json.dumps(conf, indent=4))

    os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
    device = torch.device(0)

    model_name = conf['model']['name']
    optimizer_name = conf['optimizer']['name']
    data_set = conf['data']['dataset']
    scheduler_name = conf['scheduler']['name']
    loss = get_loss(**conf['loss'])
    print("aaaa", device)

    loss.to(device)


    support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                           Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])
    model, trainer = create_model(model_name,
                                  loss,
                                  conf['model'][model_name],
                                  data_category,
                                  device,
                                  support)

    optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
    scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
    if torch.cuda.device_count() > 1:
        print("use ", torch.cuda.device_count(), "GPUS")
        model = nn.DataParallel(model)
    else:
        model.to(device)

    save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
    run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

    shutil.rmtree(save_folder, ignore_errors=True)
    os.makedirs(save_folder)
    shutil.rmtree(run_folder, ignore_errors=True)
    os.makedirs(run_folder)

    with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
        yaml.safe_dump(conf, _f)

    data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                          model_name=model_name)


    train_model(model=model,
                       dataloaders=data_loader,
                       trainer=trainer,
                       optimizer=optimizer,
                       normal=normal,
                       scheduler=scheduler,
                       folder=save_folder,
                       tensorboard_folder=run_folder,
                       device=device,
                       **conf['train'])
    test_model(folder=save_folder,
                      trainer=trainer,
                      model=model,
                      normal=normal,
                      dataloaders=data_loader,
                      device=device)


In [3]:
# print(conf['taxi'])

In [4]:
con = 'evoconv2-config'
data = ['bike']
with open(os.path.join('config', f'{con}.yaml')) as f:
    conf = yaml.safe_load(f)
# train(conf, data)

In [5]:
# conf
data_category=['taxi']

os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
device = torch.device(0)

model_name = conf['model']['name']
optimizer_name = conf['optimizer']['name']
data_set = conf['data']['dataset']
scheduler_name = conf['scheduler']['name']
loss = get_loss(**conf['loss'])
print("aaaa", device)

loss.to(device)


support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                       Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])
model, trainer = create_model(model_name,
                              loss,
                              conf['model'][model_name],
                              data_category,
                              device,
                              support)

optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
if torch.cuda.device_count() > 1:
    print("use ", torch.cuda.device_count(), "GPUS")
    model = nn.DataParallel(model)
else:
    model.to(device)

save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

shutil.rmtree(save_folder, ignore_errors=True)
os.makedirs(save_folder)
shutil.rmtree(run_folder, ignore_errors=True)
os.makedirs(run_folder)

with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
    yaml.safe_dump(conf, _f)

data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                      model_name=model_name)






aaaa cuda:0
std: 37.78643496676519 mean: 26.451628298080365
nodevec1 : torch.Size([266, 50])
nodevec2 : torch.Size([50, 266])
w1 : torch.Size([50, 50])
w2 : torch.Size([50, 50])
b1 : torch.Size([50])
b2 : torch.Size([50])
encoder.0.ru_gate_g_conv.graphconv.0.out.weight : torch.Size([50, 108])
encoder.0.ru_gate_g_conv.graphconv.0.out.bias : torch.Size([50])
encoder.0.ru_gate_g_conv.graphconv.1.out.weight : torch.Size([50, 200])
encoder.0.ru_gate_g_conv.graphconv.1.out.bias : torch.Size([50])
encoder.0.ru_gate_g_conv.graphconv.2.out.weight : torch.Size([50, 200])
encoder.0.ru_gate_g_conv.graphconv.2.out.bias : torch.Size([50])
encoder.0.ru_gate_g_conv.attlinear.weight : torch.Size([1, 13300])
encoder.0.ru_gate_g_conv.attlinear.bias : torch.Size([1])
encoder.0.candidate_g_conv.graphconv.0.out.weight : torch.Size([25, 108])
encoder.0.candidate_g_conv.graphconv.0.out.bias : torch.Size([25])
encoder.0.candidate_g_conv.graphconv.1.out.weight : torch.Size([25, 100])
encoder.0.candidate_g_conv.

In [6]:
# print(data_loader)

In [7]:
# print(data_loader)

In [8]:
train_model(model=model,
                   dataloaders=data_loader,
                   trainer=trainer,
                   optimizer=optimizer,
                   normal=normal,
                   scheduler=scheduler,
                   folder=save_folder,
                   tensorboard_folder=run_folder,
                   device=device,
                   **conf['train'])

path does not exist
EvoNN2(
  (encoder): DCRNNEncoder(
    (0): DCGRUCell(
      (ru_gate_g_conv): EvolutionCell(
        (perceptron): ModuleList()
        (graphconv): ModuleList(
          (0): GraphConv_(
            (out): Linear(in_features=108, out_features=50, bias=True)
          )
          (1): GraphConv_(
            (out): Linear(in_features=200, out_features=50, bias=True)
          )
          (2): GraphConv_(
            (out): Linear(in_features=200, out_features=50, bias=True)
          )
        )
        (attlinear): Linear(in_features=13300, out_features=1, bias=True)
      )
      (candidate_g_conv): EvolutionCell(
        (perceptron): ModuleList()
        (graphconv): ModuleList(
          (0): GraphConv_(
            (out): Linear(in_features=108, out_features=25, bias=True)
          )
          (1): GraphConv_(
            (out): Linear(in_features=100, out_features=25, bias=True)
          )
          (2): GraphConv_(
            (out): Linear(in_features=10

train epoch:   0, train loss: 29.2323: : 94it [00:25,  3.66it/s]
validate epoch:   0, validate loss: 26.5654: : 21it [00:01, 17.45it/s]


Better model at epoch 0 recorded.


test  epoch:   0, test  loss: 26.0422: : 21it [00:01, 16.81it/s]
train epoch:   1, train loss: 18.8065: : 94it [00:25,  3.68it/s]
validate epoch:   1, validate loss: 22.0623: : 21it [00:01, 17.73it/s]


Better model at epoch 1 recorded.


test  epoch:   1, test  loss: 21.2594: : 21it [00:01, 16.53it/s]
train epoch:   2, train loss: 15.4374: : 94it [00:25,  3.66it/s]
validate epoch:   2, validate loss: 23.4279: : 21it [00:01, 16.47it/s]
test  epoch:   2, test  loss: 22.6942: : 21it [00:01, 15.34it/s]


In [9]:
# import os
# import numpy as np
# import math
# import json
# import torch
# from tensorboardX import SummaryWriter
# from torch import nn, optim
# from tqdm import tqdm
# import copy, time
# from utils.util import save_model, get_number_of_parameters
# from collections import defaultdict
# from utils.util import loss_calculate
# from utils.evaluate import evaluate
# from utils.util import MyEncoder

# def test_model(folder: str,
#                   trainer,
#                   model,
#                   normal,
#                   dataloaders,
#                   device):

#     save_path = os.path.join(folder, 'best_model.pkl')
#     save_dict = torch.load(save_path)
#     # model.load_state_dict(save_dict['model_state_dict'])

#     # model.eval()
#     steps, predictions, running_targets = 0, list(), list()
#     tqdm_loader = tqdm(enumerate(dataloaders['test']))
#     for step, (inputs, targets) in tqdm_loader:
#         running_targets.append(targets.numpy())

#         with torch.no_grad():
#             inputs = inputs.to(device)
#             targets = targets.to(device)
#             outputs, loss = trainer.train(inputs, targets, 'test')
#             predictions.append(outputs.cpu().numpy())

#     running_targets, predictions = np.concatenate(running_targets, axis=0), np.concatenate(predictions, axis=0)
# #     print("xx", predictions)

#     scores = evaluate(running_targets, predictions,normal)

#     print('test results:')
#     print(json.dumps(scores,cls=MyEncoder, indent=4))
#     with open(os.path.join(folder, 'test-scores.json'), 'w+') as f:
#         json.dump(scores, f,cls=MyEncoder, indent=4)
#     if trainer.model.graph0 is not None:
#         np.save(os.path.join(folder, 'graph0'),trainer.model.graph0.detach().cpu().numpy())
#         np.save(os.path.join(folder, 'graph1'),trainer.model.graph1.detach().cpu().numpy())
#         np.save(os.path.join(folder, 'graph2'),trainer.model.graph2.detach().cpu().numpy())

#     np.savez(os.path.join(folder, 'test-results.npz'), predictions=predictions, targets=running_targets)


In [10]:
test_model(folder=save_folder,
                  trainer=trainer,
                  model=model,
                  normal=normal,
                  dataloaders=data_loader,
                  device=device)

21it [00:01, 20.12it/s]


test results:
{
    "MAE": {
        "horizon-0": 7.551253336162583,
        "horizon-1": 7.996389692114992,
        "horizon-2": 8.658520477697394,
        "horizon-3": 9.474710027928557,
        "horizon-4": 10.432249475860072,
        "horizon-5": 11.378130167022558,
        "horizon-6": 12.374610966163676,
        "horizon-7": 13.356435267734145,
        "horizon-8": 14.346359775760678,
        "horizon-9": 15.28544093312896,
        "horizon-10": 16.20248160102719,
        "horizon-11": 17.133370209811034
    },
    "RMSE": {
        "horizon-0": 15.756126779429646,
        "horizon-1": 16.58594580258417,
        "horizon-2": 17.48982996943608,
        "horizon-3": 18.649533363343604,
        "horizon-4": 19.977532538460537,
        "horizon-5": 21.355691507261053,
        "horizon-6": 22.76119275504853,
        "horizon-7": 24.080624382449948,
        "horizon-8": 25.455666241356067,
        "horizon-9": 26.899228211763038,
        "horizon-10": 28.428395839667164,
        "horiz

In [11]:
import pandas as pd
import numpy as np
a=pd.read_pickle('data/adj_data/adj_mx_taxi.pkl')

In [12]:
a=np.array(a)


/home/lambda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [13]:
print(type(a[2]))
# # print(a[1])
# print((a[1].items()))


<class 'numpy.ndarray'>


In [14]:
# 256*256

In [15]:
print(dir(data_set))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


In [16]:
xx=data_loader['test']

In [17]:
print(type(xx))

<class 'torch.utils.data.dataloader.DataLoader'>


In [18]:
for i, batch in enumerate(xx):
    aa=batch

In [19]:
aa=np.array(aa)

/home/lambda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  """Entry point for launching an IPython kernel.
/home/lambda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [20]:
print(aa[0].shape)

torch.Size([32, 12, 266, 2])


In [21]:
from tqdm import tqdm


In [22]:
tqdm_loader = tqdm(enumerate(data_loader['test']))
for step, (inputs, targets) in tqdm_loader:
#     print(step)
    x=inputs 
    y=targets

21it [00:00, 190.90it/s]


In [23]:
print(x[0].shape)
print(y[0].shape)
print(x[0][0][1], y[0][0][2])

torch.Size([12, 266, 2])
torch.Size([12, 266, 2])
tensor([2.1581, 4.9634]) tensor([-0.5677, -0.5942])


In [24]:
# 7. convert the adj  matrix to adjacency list
edge_list=A_to_edge_index(adj2)
print(edge_list)

NameError: name 'A_to_edge_index' is not defined